In [ ]:
#necessary liberaries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import zipfile
import pathlib
import tensorflow as tf
import glob
import os
from PIL import Image
import math
from sklearn.model_selection import train_test_split

In [ ]:
pip install opencv-python

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [ ]:
#Getting data
os.listdir('C:/Users/User/Downloads/dataset/data')

In [ ]:
new_data = []
imgs = 'C:/Users/User/Downloads/dataset/data'
for img in (os.listdir(imgs)):
  image = Image.open(img)
  data = asarray(image)
  image1 = Image.fromarray(data)
  img_array = cv2.imread(image1)
  new = cv2.resize(img,(512,512))
  new_data.append(new)
return new_data

In [ ]:
#data splitting
x_train =new_data()[:2500]
x_test = np.random.shuffle(new_data()[:1500])

In [ ]:
#data resizing
x_train = np.reshape(x_train, (len(x_train), 512, 512, 3)) / 255.
x_test = np.reshape(x_test, (len(x_test), 512, 512, 3)) / 255.


In [ ]:
#Model Training
model = Sequential()

input_shape = (512, 512, 3)
# Encoder layers
model.add(Conv2D(16, (3, 3), activation='relu', padding='same',input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))

model.add(Conv2D(8, (3, 3), activation='sigmoid', padding='same'))
#decoder layers
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
epoch = 30
batch_size = 64
train_batch_size = 1000
train_batch_per_epoch = math.ceil(x_train.shape[0] / train_batch_size)

In [ ]:
#from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=7, init ='k-means++', max_iter=300, n_init=10,random_state=0 )
kmeans1 = kmeans.fit_predict(x_test)

In [ ]:
#visualisation
plt.scatter(x_test[kmeans1==0, 0], x_test[kmeans1==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(x_test[kmeans1==1, 0], x_test[kmeans1==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(x_test[kmeans1==2, 0], x_test[kmeans1==2, 1], s=100, c='green', label ='Cluster 3')
plt.scatter(x_test[kmeans1==3, 0], x_test[kmeans1==3, 1], s=100, c='cyan', label ='Cluster 4')
plt.scatter(x_test[kmeans1==4, 0], x_test[kmeans1==4, 1], s=100, c='magenta', label ='Cluster 5')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='yellow', label = 'Centroids')
plt.show()

In [ ]:
history=model.fit(x_train, x_train,epochs=1,batch_size=batch_size,shuffle=True,validation_data=(x_test[0:200], x_test[0:200]))

In [ ]:
#result visualisation
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('training loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
converted_imgs = model.predict(x_test)
encoder = Model(inputs=model.input, outputs=model.get_layer('code').output)
encoder.summary()

In [ ]:
# encoder to get the 'codes' for each image
coded_imgs = encoder.predict(x_test)
coded_imgs[0]

In [ ]:
# Use the 'cosine similarity' as a measure 
# each of the first ten image codes
similar = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        rel = cosine(coded_imgs[i], coded_imgs[j])
        similar[i,j] = rel
ax = sns.heatmap(sim)

In [ ]:
img_idx = 90
plt.imshow(x_test[img_idx].reshape(28,28))
src_img_code = coded_imgs[img_idx]

In [ ]:

def find_similar_images(idx):
    src_img_code = coded_imgs[idx]

    # calculate the cosine similarity from our reference image to all the other image codes
    similarities = [ (cosine(src_img_code, coded_imgs[i]), i) for i in range(len(coded_imgs)) ]
    similarities.sort()
    similar_idxs = [ i for _,i in similarities ]

    # plot top 10 matches
    plt.figure(figsize=(20,4))
    for i, idx in enumerate(similar_idxs[:n]):
        ax = plt.subplot(2, n, i + 1)
        ax.axis('off')
        if i == 0:
          ax.set_title("Original")
        else:
          ax.set_title(idx)
        plt.imshow(x_test[idx].reshape(28,28))


for i in [10, 20, 30, 40, 50, 60]:
    find_similar_images(i)
    plt.show()